In [1]:
import pandas as pd
import dash
from dash import dcc, html
import altair as alt
from dash.dependencies import Input, Output

df = pd.read_csv('your_output_file.csv')
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race,Education,latitude,longitude
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,1,0,0,0,0,0,NaN,NaN,37.486324,-122.232523
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,2,0,0,0,0,0,NaN,NaN,37.779259,-122.419329
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,3,0,0,0,0,0,NaN,NaN,47.603832,-122.330062
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,7,0,0,0,0,0,NaN,NaN,37.368830,-122.036349
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,9,0,0,0,0,0,NaN,NaN,37.389389,-122.083210


In [2]:
app = dash.Dash(__name__)

alt.data_transformers.disable_max_rows()

app.layout = html.Div([
    html.H1("Top 10 Companies by Average Salary"),
    html.Div([
        html.Iframe(
            id='bar-chart',
            style={'width': '50%', 'height': '500px', 'display': 'inline-block'}
        ),
        html.Iframe(
            id='pie-chart',
            style={'width': '50%', 'height': '500px', 'display': 'inline-block'}
        )
    ])
])

@app.callback(
    [Output('bar-chart', 'srcDoc'),
     Output('pie-chart', 'srcDoc')],
    [Input('bar-chart', 'id')]
)
def update_plots(_):
    # Bar chart
    filtered_df = df[df['totalyearlycompensation'] > 0]
    avg_salary_by_company = filtered_df.groupby('company')['totalyearlycompensation'].mean().reset_index()
    top_10_companies = avg_salary_by_company.nlargest(10, 'totalyearlycompensation')
    
    bar_chart = alt.Chart(top_10_companies).mark_bar().encode(
        x=alt.X('company:N', 
                sort='-y',
                axis=alt.Axis(labelAngle=-45, title='Company')),
        y=alt.Y('totalyearlycompensation:Q',
                title='Average Yearly Compensation ($)'),
        color=alt.Color('company:N', legend=None)
    ).properties(
        width=400,
        height=400,
        title='Top 10 Companies by Average Salary'
    )

    # Pie chart
    # Map gender values to simplified categories
    df['gender_category'] = df['gender'].map(lambda x: 'male' if str(x).lower() in ['m', 'male'] 
                                           else ('female' if str(x).lower() in ['f', 'female'] 
                                           else 'other'))
    
    gender_counts = df['gender_category'].value_counts().reset_index()
    gender_counts.columns = ['gender', 'count']
    
    pie_chart = alt.Chart(gender_counts).mark_arc().encode(
        theta='count:Q',
        color=alt.Color('gender:N', scale=alt.Scale(domain=['male', 'female', 'other'])),
        tooltip=['gender:N', 'count:Q']
    ).properties(
        width=400,
        height=400,
        title='Gender Distribution'
    )

    return bar_chart.to_html(), pie_chart.to_html()

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)